# Calculate lifting condensation level (LCL)

LCL calculated using standard procedure following G. S. Stipanuk, 1973. 

Elzina Bala, elzinabala@gmail.com

In [1]:
import xarray as xr
import dask
import shutil
import numpy as np


In [2]:
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:36243 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 6 Cores: 24 Memory: 134.22 GB


Load dictionary will all simulations

In [4]:
import sys
sys.path.append('/pf/b/b380459/nawdex-hackathon/')
import dict_nawdexsims

simdict   = dict_nawdexsims.simdictionary()

ModuleNotFoundError: No module named 'dict_nawdexsims'

In [ ]:
# constants declaration for lifitng condesation level

TNOT = 273.15
RCP = 0.286
ES0 = 6.1121
C1 = 0.0498646455
C2 = 2.4082965
C3 = 7.07475
C4 = 38.9114
C5 = 0.0915
C6 = 1.2035

# constants declaration for adiabatic laps rate

Lv  = 2470000. #; J kg-1
Cp  =  1015.   #; J kg K-1
gr  =  9.8     #; m/s2
Rv  = 461.5    #; J/kg K
Ra  = 286.9    #; J/kg K


In [ ]:
def load_data(expid):
    
    path  = '/scratch/b/b380459/icon_4_hackathon/'
    fname = path+'/'+expid+'/'+expid+'_2016*_fg_DOM01_ML_*.nc'
    ds = ( xr.open_mfdataset(fname, combine='by_coords',parallel=True, 
                             engine='h5netcdf', chunks={'time': 1})
           [['temp', 'pres', 'td_2m', 'z_ifc']].rename({'ncells_2': 'ncells'}) )

    return ds

Note: Level with index 17 is at 1.5km height above ocean, Level with index 52 is at 3km height above ocean, level with index 74 is the surface. We use a fixed height level here because we intend to only look at the ocean points later.

zLCL calculated based on Stipanuk, G.S., (1973) original version."Algorithms for generating a skew-t, log p diagram and computing selected meteorological quantities."

In [ ]:

def ZLCL(TK,TDK,P):
    ### unit conversion from K to C
    TC = TK - TNOT
    TDC = TDK - TNOT

    ## determine the mixing ratio line through td and p. 
    ES = ES0*np.exp(17.67*TDC/ (TDC+243.5))

    ## saturation mix ratio ##same as qs
    WS = 622.*ES/ (P-ES)

    ## determine the dry adiabat through t and p (ie: potential temperature)
    PTK = TK* ((1000./P)**RCP)

    ## iterate to locate pressure pi at the intersection of the two curves. 
    ## pi has been set to p for the initial guess.
    ## tdac - temp [C] on a dry adiabat at p where the dry adibat is given by the potential temperature.
    ## tmrc - temperature on a mix ratio line [ws] at pressure p 
    TDAC = PTK* ((P/1000.)**RCP) - TNOT
    Z = np.log10(WS*P/ (622.+WS))
    TMRK = 10.** (C1*Z+C2) - C3 + C4* ((10.** (C5*Z)-C6)**2.)
    TMRC = TMRK - TNOT
    X = 0.02* (TMRC-TDAC)
    PLCL = P.where(np.abs(X) < 0.01, P* 2.**(X))

    ZLCL = (Ra/gr)*TK*np.log((P/PLCL))
    return ZLCL,WS


In [ ]:
## load TK (temperature) and TDK (dew point temperature)
#T850 = load_data('nawdexnwp-80km-mis-0001')['temp'].isel(height=17) ### temperature at 850hPa
#TK = load_data('nawdexnwp-80km-mis-0001')['temp'].isel(height=74) ### temperature at surface
#TDK = load_data('nawdexnwp-80km-mis-0001')['td_2m'] ###dew point temperature at surface
#P = load_data('nawdexnwp-80km-mis-0001')['pres'].isel(height=74) ### pressure at surface
#z700 = load_data('nawdexnwp-80km-mis-0001')['z_ifc'].isel(height=52) ### height at 700hPa=3km


In [ ]:
for sim in list(simdict.keys()):
    print('Working on:', sim)
    ds = load_data(sim)
    #ds = ds.isel(height=[17,52,74])
    
    ds['theta'] = ds['temp'] * (100000./ds['pres'])**(Ra/Cp)
    ds['lts'] = (ds['theta'].isel(height=74) - ds['theta'].isel(height=52) ) / 3000.
    del ds['theta']
    
    T850 = ds['temp'].isel(height=17) ### temperature at 850hPa
    
    zlcl,_ = ZLCL(ds['temp'].isel(height=74), ds['td_2m'], ds['pres'].isel(height=74))
    _,WS = ZLCL(ds['temp'].isel(height=74), ds['td_2m'], ds['pres'].isel(height=74))
    
    Gamma = (gr/Cp)*(1-(1+Lv*WS/Ra/(T850+273.15))/(1+Lv*Lv*WS/Cp/Rv/(T850+273.15)/(T850+273.15)))
    del WS, ds['temp'], ds['td_2m'], ds['pres']
    
    z700 = ds['z_ifc'].isel(height_3=52) ### height at 700hPa=3km

    ds['eis'] = ds['lts'] - Gamma * (z700 - zlcl)
    

    ds = ds.drop(['temp', 'pres', 'td_2m', 'z_ifc','lts'])
    ds.attrs['units'] = 'Kelvin m-1'
    ds.attrs['description'] = 'estimated inversion strength according to Stipanuk, G.S., 1973, doi: 10.21236/ad0769739'
    ds.attrs['simulation'] = sim
    # store to zarr store
    zarr_store = '/scratch/b/b380459/icon_4_hackathon/temp/eis_'+sim+'.zarr'
    # remove any zarr_store with same name that might have been created previously
    shutil.rmtree(zarr_store, ignore_errors=True)
    ds.to_zarr(zarr_store)
    

Clean up

In [ ]:
client.close()